In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import json
import numpy as np
from fastai.vision.all import *
import albumentations as A
import skimage.io as skio
import warnings
warnings.filterwarnings("ignore")

In [ ]:
with open('./data/BigEarthNet-S2_19-classes_models/label_indices.json', 'rb') as f:
    label_indices = json.load(f)
label_conversion = label_indices['label_conversion']
BigEarthNet_19_label_idx = {v: k for k, v in label_indices['BigEarthNet-19_labels'].items()}
def get_label(patch_json):
    original_labels = patch_json['labels']
    original_labels_multi_hot = np.zeros(
    len(label_indices['original_labels'].keys()), dtype=int)
    BigEarthNet_19_labels_multi_hot = np.zeros(len(label_conversion),dtype=int)
    for label in original_labels:
        original_labels_multi_hot[label_indices['original_labels'][label]] = 1

    for i in range(len(label_conversion)):
        BigEarthNet_19_labels_multi_hot[i] = (
                np.sum(original_labels_multi_hot[label_conversion[i]]) > 0
            ).astype(int)

    BigEarthNet_19_labels = ''
    for i in np.where(BigEarthNet_19_labels_multi_hot == 1)[0]:
        BigEarthNet_19_labels+=str(i)+' '
    return BigEarthNet_19_labels[:-1]


In [ ]:
# # create data
# df=pd.read_csv('./data/BigEarthNet-S2_19-classes_models/splits/train.csv',header=None)
# df['Isval']=0
# df2=pd.read_csv('./data/BigEarthNet-S2_19-classes_models/splits/val.csv',header=None)
# df2['Isval']=1
# df=pd.concat([df,df2])
# df=df.rename(columns={0: "fname"})
# df['label']=''
# for i in tqdm(range(len(df))):
#     with open('./data/BigEarthNet-v1.0/'+df.iat[i,0]+'/'+df.iat[i,0]+'_labels_metadata.json', 'rb') as f:
#         patch_json = json.load(f)
#     df.iat[i,2]=get_label(patch_json)  
#     df.iat[i,0]='./data/BigEarthNet-v1.0/'+df.iat[i,0]+'/'+df.iat[i,0]+'.tif'  

In [ ]:
#export
def open_tif(fn, cls=torch.Tensor):
    im = skio.imread(str(fn))/10000
    im = im.transpose(1,2,0).astype('float32')
    return cls(im)
class MSTensorImage(TensorImage):
    
    @classmethod
    def create(cls, data:(Path,str,ndarray), chnls=None):
        
        if isinstance(data, Path) or isinstance(data, str):
            if str(data).endswith('tif'): im = open_tif(fn=data,cls=torch.Tensor)

        elif isinstance(data, ndarray): 
            im = torch.from_numpy(data)
        else:
            im = data
        
        return cls(im)

In [ ]:
df=pd.read_csv('./data/file.csv')

In [ ]:
df.head()

,fname,Isval,label
0,./data/BigEarthNet-v1.0/S2A_MSIL2A_20170717T113321_28_87/S2A_MSIL2A_20170717T113321_28_87.tif,0,4
1,./data/BigEarthNet-v1.0/S2A_MSIL2A_20170717T113321_28_90/S2A_MSIL2A_20170717T113321_28_90.tif,0,4
2,./data/BigEarthNet-v1.0/S2A_MSIL2A_20170717T113321_29_83/S2A_MSIL2A_20170717T113321_29_83.tif,0,4
3,./data/BigEarthNet-v1.0/S2A_MSIL2A_20170717T113321_29_84/S2A_MSIL2A_20170717T113321_29_84.tif,0,4
4,./data/BigEarthNet-v1.0/S2A_MSIL2A_20170717T113321_29_85/S2A_MSIL2A_20170717T113321_29_85.tif,0,4


In [ ]:
db=DataBlock(blocks=(TransformBlock(type_tfms=partial(MSTensorImage.create)), MultiCategoryBlock),
                   splitter=ColSplitter('Isval'),
                   get_x=ColReader('fname'),
                   get_y=ColReader('label', label_delim=' '))
#                    batch_tfms=aug_transforms(size=224))
# db.summary(source=df)

In [ ]:
ds = db.datasets(source=df)

In [ ]:
#export
BAND_STATS = {
            'S2':{
                'mean': {
                    'B01': 340.76769064,
                    'B02': 429.9430203,
                    'B03': 614.21682446,
                    'B04': 590.23569706,
                    'B05': 950.68368468,
                    'B06': 1792.46290469,
                    'B07': 2075.46795189,
                    'B08': 2218.94553375,
                    'B8A': 2266.46036911,
                    'B09': 2246.0605464,
                    'B11': 1594.42694882,
                    'B12': 1009.32729131
                },
                'std': {
                    'B01': 554.81258967,
                    'B02': 572.41639287,
                    'B03': 582.87945694,
                    'B04': 675.88746967,
                    'B05': 729.89827633,
                    'B06': 1096.01480586,
                    'B07': 1273.45393088,
                    'B08': 1365.45589904,
                    'B8A': 1356.13789355,
                    'B09': 1302.3292881,
                    'B11': 1079.19066363,
                    'B12': 818.86747235
                }
            },
            'S1': {
                'mean': {
                    'VV': -12.619993741972035,
                    'VH': -19.29044597721542,
                    'VV/VH': 0.6525036195871579,
                },
                'std': {
                    'VV': 5.115911777546365,
                    'VH': 5.464428464912864,
                    'VV/VH': 30.75264076801808,
                },
                'min': {
                    'VV': -74.33214569091797,
                    'VH': -75.11137390136719,
                    'R': 3.21E-2
                },
                'max': {
                    'VV': 34.60696029663086,
                    'VH': 33.59768295288086,
                    'R': 1.08
                }
            }
        }


In [ ]:
#export
bands=['B02','B03', 'B04', 'B05','B06', 'B07', 'B11', 'B08','B8A', 'B12']

In [ ]:
#export
means=[BAND_STATS['S2']['mean'][band]/10000 for band in bands]
stds=[BAND_STATS['S2']['std'][band]/10000 for band in bands]

In [ ]:
#export 
# Now we will create a pipe of transformations
from albumentations.pytorch import ToTensorV2
aug_pipe = A.Compose([A.ShiftScaleRotate(p=.5),
                      A.HorizontalFlip(),
                      A.Normalize(mean=means,std=stds,max_pixel_value=1.0),
                         ToTensorV2()]                    
                    )
val_pipe = A.Compose([
    A.Normalize(mean=means,std=stds,max_pixel_value=1.0),
                         ToTensorV2()]                    
                    )

class TrainTransform(ItemTransform):
    split_idx = 0
    def __init__(self, aug,split=0): 
        self.aug = aug
#         self.split_idx = split
    def encodes(self, x):
        aug = self.aug(image=x[0].numpy())
#         print(torch.cat((aug['image0'],aug['image1']),axis=0).shape)
        return aug['image'], x[1]
class ValTransform(ItemTransform):
    split_idx = 1
    def __init__(self, aug,split=0): 
        self.aug = aug
#         self.split_idx = split
    def encodes(self, x):
        aug = self.aug(image=x[0].numpy())
#         print(torch.cat((aug['image0'],aug['image1']),axis=0).shape)
        return aug['image'], x[1]

# Create our class with this aug_pipe
aug = TrainTransform(aug_pipe)
aug2=ValTransform(val_pipe)

In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(MSTensorImage.create)), MultiCategoryBlock),
                   splitter=ColSplitter('Isval'),
                   get_x=ColReader('fname'),
                   get_y=ColReader('label', label_delim=' '),
                   item_tfms=[aug,aug2]
              )

In [ ]:
dls = db.dataloaders(source=df, bs=2, num_workers=0)

In [ ]:
aa,bb=first(dls.train)

In [ ]:
aa.min()

tensor(-1.6713, device='cuda:0')

In [ ]:
from nbdev.export import notebook2script


In [ ]:
notebook2script(fname='./00_core.ipynb')

Converted 00_core.ipynb.
